# Trabalho Final - Processamento de Linguagem Natural

**Professor**: Anderson Dourado

**Turma**: 10IA

**Integrantes**:

    Carlos Eduardo Barbosa - 335518
    Daniel Gregoris Guarino - 335398
    Fabio de Campos Bordin - 336263
    Fernando Bareno Calo - 335434
    
**Data**: 30/05/2020

### Classificador de Sentimentos

Este notebook é o trabalho de criação de um modelo de machine learning que funcione como um **classificador de sentimentos** de crítica de filmes usando técnicas de **Processamento de Linguagem Natural**. As críticas dos filmes foram obtida do site **imdb (Internet Movie Database)** a princípio na lingua inglesa, porém, o classificador deste trabalho deverá funcionar com textos em português, a versão neste idioma já consta no dataset que iremos utilizar. Os sentimentos em relação aos filmes podem ser classificados como **positivo** ou **negativo**. O modelo deverá ser avaliado conforme a métrica F1 Score, que é a média ponderada dos índices **Recall** e **Precisão**, 
F1 = $$\begin{equation*} 
\frac{2*precision*recall}{precision+recall} sendo a resposta um número entre 0 e 1, sendo que quanto mais perto de 1, mais preciso é o modelo (com os devidos cuidados em relação a overfitting, é claro). No desenvolvimento deste notebook serão realizados diversos experimentos com o intuito de se obter o melhor F1 Score para o classificador.





In [ ]:
# instalar biblioteca unidecode
!pip install unidecode

     |████████████████████████████████| 245kB 2.7MB/s 


In [ ]:
# bibliotecas
import os
import csv
import requests
from unidecode import unidecode
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook as tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.rslp import RSLPStemmer
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# download do arquivo do dataset
url = 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/imdb-reviews-pt-br.csv'

def download_file(url):
    request_url = requests.get(url)
    if request_url.status_code == requests.codes.OK:
        with open('imdb-reviews-pt-br.csv', 'wb') as file:
            file.write(request_url.content)
    else:
        resposta.raise_for_status()
        
download_file(url)

In [ ]:
# conferir arquivo do dataset no diretório
% ls

imdb-reviews-pt-br.csv  sample_data/


In [ ]:
# abrir e visualizar dataset no Pandas
df = pd.read_csv('imdb-reviews-pt-br.csv')
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


### Pre Processing

In [ ]:
# remover a coluna 'id' do dataset
df.drop(labels='id',axis=1, inplace=True)
df.head()

,text_en,text_pt,sentiment
0,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
# a classificação de sentimentos será feita na lingua portuguesa, 
# portanto podemos retirar a feature com texto em inglês e trocar 
# o nome das feature 'sentiment' e 'text_pt' para a língua portuguesa também
df.drop(labels='text_en', axis=1, inplace=True)
df.rename(columns={'sentiment': 'sentimento', 'text_pt': 'texto_pt'}, inplace=True)

# conferir visualmente se o dataset possui uma coluna a menos e o novo nome
# das colunas
df.head()

,texto_pt,sentimento
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg


In [ ]:
# padronizar os textos em caixa baixa 
df['texto_pt'] = df.texto_pt.str.lower()
df.texto_pt.head()

0    mais uma vez, o sr. costner arrumou um filme p...
1    este é um exemplo do motivo pelo qual a maiori...
2    primeiro de tudo eu odeio esses raps imbecis, ...
3    nem mesmo os beatles puderam escrever músicas ...
4    filmes de fotos de latão não é uma palavra apr...
Name: texto_pt, dtype: object

In [ ]:
# padronizar texto substituindo palavras acentuadas por não acentuadas
# exemplo: latão vira latao, é vira e
df['texto_pt'] = df.texto_pt.apply(lambda text: unidecode(text))
df.texto_pt.head()

0    mais uma vez, o sr. costner arrumou um filme p...
1    este e um exemplo do motivo pelo qual a maiori...
2    primeiro de tudo eu odeio esses raps imbecis, ...
3    nem mesmo os beatles puderam escrever musicas ...
4    filmes de fotos de latao nao e uma palavra apr...
Name: texto_pt, dtype: object

In [ ]:
# verificação da quantidade de cada valor da feature alvo 
# pos (positivo) e neg (negativo)
df.sentimento.value_counts()

neg    24765
pos    24694
Name: sentimento, dtype: int64

In [ ]:
# Agora vamos avaliar a proporção de cada tipo de avaliação
negative = df.sentimento.value_counts()[0]
positive = df.sentimento.value_counts()[1]
total = negative + positive

print('Proporção de avaliação neg: %.2f%% \nProporção de avaliação pos: %.2f%% ' % (negative/total*100, positive/total*100))

Proporção de avaliação neg: 50.07% 
Proporção de avaliação pos: 49.93% 


O dataset está balanceado, temos metade dos filmes avaliados como negativo e a outra metade como positivo, é necessário **manter essa proporção dos dados no momento da validação cruzada** (separar o dataset em conjunto de treino, validação e teste).

In [ ]:
# o sklearn, biblioteca que usaremos para construir o classificador de sentimentos não consegue realizar calculos
# se tivermos features do tipo string. Portanto mudaremos os valores da feature alvo de 'pos' para 1 e 'neg' para 0

target_encode = {'neg':0, 'pos': 1 }

df.sentimento.replace(target_encode, inplace=True)

df.sentimento.value_counts()

0    24765
1    24694
Name: sentimento, dtype: int64

In [ ]:
# salvar um novo arquivo .csv do dataset com o pré processamento aplicado acima
df.to_csv('imdb-reviews-pt-br-modified.csv', index=False)

Conclusões de Pre Processing: Este pré processamento é realizado para que os dados estejam preparados para as técnicas de NLP que serão aplicadas 

### Funções auxiliares a modelagem do classificador de sentimentos

In [ ]:
# checar se o módulo nltk está instalado
def check_module(module):
    nltk.download(module)

# configurar stopwords da biblioteca nltk para a lingua especificada
def setup_nltk_stopwords(language, module):
    check_module(module)   
    return nltk.corpus.stopwords.words(language)

In [ ]:
# tokenização - necessário para aplicação de STEMMER usando a biblioteca nltk
# esta biblioteca ao contrario do sklearn nao faz a tokenização automaticamente
def tokenize(feature,module):
    check_module(module)
    data_frame = feature
    return data_frame.apply(word_tokenize)    

In [ ]:
# Contagem de termos com n-gramas usando o método CountVectorizer 
def count_term(ngram=None,words=None):
    return CountVectorizer(ngram_range=ngram, stop_words=words)

# TF-IDF
def tf_idf(ngram=None, words=None, idf=False):
    return TfidfVectorizer(ngram_range=ngram, use_idf=idf, stop_words=words)

# treinar
def vect_fit(feature, vect):
    return vect.fit(feature)

# transformar - para dataset de teste usar apenas vect_transform
def vect_transform(feature, vect_fit):
    return vect.transform(feature)    


In [ ]:
# Stemmer
def init_rslp():
    check_module('rslp')
    return RSLPStemmer()

def stem_pandas(line):
    return ' '.join([rslp.stem(token) for token in line])

In [ ]:
# score básico (como é calculado?)
def accuracy(y_test, y_pred):
    return {'Acurácia: ': accuracy_score(y_test, y_pred)}

# matriz de confusão
def confusion(classifier, y_test, y_pred):
    return {'Matriz de Confusão: ' : confusion_matrix(y_test, y_pred)}
    
# f1 score
def f1(y_test, y_pred):
    return {'F1 Score' : f1_score(y_test, y_pred)}

In [ ]:
# Classificador com modelo de Árvore de Decisão - DecisionTree 
descricao = 'descrição muito breve da configuração do modelo'
score_dict = {}

def decision_tree_classifier(X_train, y_train, X_test,y_test):
       
    classifier = DecisionTreeClassifier()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    score_ = accuracy(y_test, y_pred)
    conf_ = confusion(classifier, y_test, y_pred)
    f1_ = f1(y_test, y_pred)
    
    print('Árvore de Decisão')
    print(score_)
    print(conf_)
    print(f1_)
    

In [ ]:
# modelo Regressão Logística
def logistic_regression_classifier(X_train, y_train, X_test,y_test):
       
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    score_ = accuracy(y_test, y_pred)
    conf_ = confusion(classifier, y_test, y_pred)
    f1_ = f1(y_test, y_pred)
    
    print('Regressão Logística')
    print(score_)
    print(conf_)
    print(f1_)


In [ ]:
# modelo SVM
def svm_classifier(X_train, y_train, X_test, y_test):
    
    classifier = svm.SVC()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    score_ = accuracy(y_test, y_pred)
    conf_ = confusion(classifier, y_test, y_pred)
    f1_ = f1(y_test, y_pred)
    
    print('SVM')
    print(score_)
    print(f1_)

In [ ]:
# modelo naïve bayes
def naive_bayes_classifier(X_train, y_train, X_test, y_test, alpha):
    
    classifier = MultinomialNB(alpha=alpha)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    score_ = accuracy(y_test, y_pred)
    conf_ = confusion(classifier,  y_test, y_pred)
    f1_ = f1(y_test, y_pred)
    
    print('Naïve Bayes')
    print(score_)
    print(f1_)

In [ ]:
# separar dataset entre treino e teste
def split_data(X, y, size):
    return train_test_split(X, y, test_size=size)

# quantidade de registros em cada dataset
def return_shape(train, val, test):
    return {'Train Shape': train.shape[0], 
            'Validation Shape': val.shape[0],
            'Test Shape': test.shape[0]}

# proporção de cada dataset (treino, validação e teste) em relação ao dataset completo
def proportion_train_val_test(df, train, val, test):
     
    data_size = []
    data_list = [train, val, test]
    full_size = df.shape[0]
    
    for fraction in data_list:
        data_size.append(fraction.shape[0]/full_size)        
        
    return {'Train Prop': data_size[0],
            'Validation Prop' : data_size[1],
            'Test Prop': data_size[2]}
                       

# proporção de respostas em cada dataset (treino, validação e teste)
def strat_train_val_test(y_train, y_val, y_test):
    target_strat = []
    data_list = [y_train, y_val, y_test]
    
    for target in data_list:
        target_strat.append(target.mean())
    
    
    return {'Train Pos': target_strat[0],
            'Validation Pos' : target_strat[1],
            'Test Pos' : target_strat[2]}
                         
                         

### Configurar semente de geração de números pseudo aleatórios

In [ ]:
# obter seed atual do numpy
np.random.get_state()[1][0]

2147483648

In [ ]:
# fixar semente (seed) para geração de números aleatórios.Com isso o default do scikit learn será o número
# definido abaixo e não teremos a necessidade de configurar isso toda vez que algum método necessitar 
# deste parametro

SEED = 42
np.random.seed(SEED)

assert np.random.get_state()[1][0] == SEED

### Configuração de validação cruzada

Para testarmos nosso modelo usaremos primeiramente o dataset de validação, para ver como o modelo se comporta ao ser alimentado com novos dados. Após os experimentos será escolhido o modelo campeão, isto é o que possuir o maior F1 Score, e só então faremos a comparação do modelo utilizando o dataset de teste

In [ ]:
# definição daproporção dos datasets de teste e validação em relação ao
# dataset completo
test_size = 0.20
val_size = 0.20

In [ ]:
# teste das funções definidas acima
# para termos uma validação cruzada robusta, dividimos o dataset de treino
# em um dataset de validação para que os modelos gerados sejam primeiro 
# conferidos em relação a esses dados de validação. Após o modelo campeão
# ser escolhido
X = df.texto_pt
y = df.sentimento

X_train, X_test, y_train, y_test = split_data(X,y, size = test_size)

X_train, X_val, y_train, y_val = split_data(X_train, y_train, size=val_size)

print(return_shape(X_train, X_val, X_test))
print(proportion_train_val_test(df, X_train, X_val, X_test))
print(strat_train_val_test(y_train, y_val, y_test))

{'Train Shape': 31653, 'Validation Shape': 7914, 'Test Shape': 9892}
{'Train Prop': 0.6399846337370347, 'Validation Prop': 0.16001132250955336, 'Test Prop': 0.2000040437534119}
{'Train Pos': 0.49830979685969734, 'Validation Pos': 0.4945665908516553, 'Test Pos': 0.5061665992721391}


Daqui para frente todas as técnicas de featuring engineering serão feitas no dataset de treino **X_train**, não podemos ter dados de validação no treino do modelo de classificação para que este não conheça de antemão os dados de validação e teste, o que é conhecido por **data leakage** e nos leve a um modelo que "decora" muito bem os dados do dataset de teste, se saindo mal quando novos valores são apresentados, evitando assim **overfitting** do modelo.

### Modelo Baseline

Nesta fase serão gerados dois modelos de classificação mais básicos possíveis com as seguintes características:

- Serão gerados apenas **unigramas**

- As 'stopwords' não serão retiradas dos texto da feature texto_pt

- **Modelo Baseline 1:** usando o dataset de validação para teste do F1 Score do modelo

- **Modelo Baseline 2:** usando um dataset com todos os valores igual 1 (avaliação positiva) para teste do F1 Score do modelo

Um desses modelo será adotado como *baseline*, ou seja, seu valor de F1 Score será o mínimo que um modelo deve alcançar, quando comparado com outros modelos.

In [ ]:
# baseline model utilizando classificação por árvore de decisão

# Pipeline de modelagem

# usar unigramas
n_gram=(1,1)

# 1) Contagem de termos com unigrama sem retirar stopwords do texto
vect = count_term(ngram=n_gram, words=None)
fit = vect_fit(X_train, vect)
vect_text = vect_transform(X_train, fit)

# 2)transformar os dados de validação mantendo as regras usadas no  
# dataset de treino
val_text = vect_transform(X_val, fit)

# 3) Treinar modelo - Árvore de decisão
# 3.1) usando dataset de validação
decision_tree_classifier(vect_text, y_train, val_text, y_val)

# 3.2) todos os filmes classificados como positivo
# qual acurácia e F1 score obtemos se todos os filmes forem classificados como positivo?
y_baseline = np.ones(y_val.shape[0])
decision_tree_classifier(vect_text, y_train, val_text, y_baseline)

Árvore de Decisão
{'Acurácia: ': 0.6988880464998737}
{'Matriz de Confusão: ': array([[2774, 1251],
       [1132, 2757]])}
{'F1 Score': 0.6982398379131316}
Árvore de Decisão
{'Acurácia: ': 0.4935557240333586}
{'Matriz de Confusão: ': array([[   0,    0],
       [4008, 3906]])}
{'F1 Score': 0.6609137055837564}


### Conclusão Baseline

Vamos adotar o modelo de classificação de árvore de decisão com **F1 Score de 69.82%** como nossa referência (baseline) ao avaliar os demais modelos gerados neste notebook.





### Modelo 1) Contagem com unigramas sem retirar stopwords com modelo de Regressão Logística

In [ ]:
# gerar modelo com a mesma configuração dos modelo anterior, com a diferença que 
# este é uma regressão logística
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Regressão Logística
{'Acurácia: ': 0.8765478898155168}
{'Matriz de Confusão: ': array([[3531,  494],
       [ 483, 3406]])}
{'F1 Score': 0.8745666966234433}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Mantendo a mesma configuração, apenas alterando o tipo de modelo já obtemos uma melhora considerável no F1 Score da **regressão logística** = 87.45% em relação à **árvore de decisão** = 69.88%

### Modelo 2) TF-IDF (unigrama com e sem stopwords) com modelos de Árvore de decisão, regresão logística, SVM.  

In [ ]:
# configurar stopwords para a língua portuguesa e acrescentando '...' a estas
# esses três pontos está presente em muitos textos deste nosso dataset 
stopwords = setup_nltk_stopwords('portuguese', 'stopwords') 
stopwords = stopwords + ['...']
stopwords[:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um']

In [ ]:
# Pipeline
# 1) TD-IDF com unigrama sem stopwords do texto
n_gram = (1,1)
vect = tf_idf(ngram=n_gram, words=stopwords, idf=True)
#vect = tf_idf(ngram=n_gram, words=None, idf=True)
fit = vect_fit(X_train, vect)
vect_text = vect_transform(X_train, fit)

# 2)transformar os dados de validação mantendo as regras usadas no  
# dataset de treino
val_text = vect_transform(X_val, fit)

In [ ]:
# 3) Treinar modelo
# 3.1) Árvore de decisão com stopwords
decision_tree_classifier(vect_text, y_train, val_text, y_val)

Árvore de Decisão
{'Acurácia: ': 0.7016679302501896}
{'Matriz de Confusão: ': array([[2828, 1172],
       [1189, 2725]])}
{'F1 Score': 0.6977339649212649}


In [ ]:
# 3.2) Regressão Logística com stopwords 
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Regressão Logística
{'Acurácia: ': 0.8953752843062927}
{'Matriz de Confusão: ': array([[3537,  463],
       [ 365, 3549]])}
{'F1 Score': 0.8955336866010597}


In [ ]:
# 3.3) Regressão Logística sem stopwords (nltk)
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Regressão Logística
{'Acurácia: ': 0.8952489259540055}
{'Matriz de Confusão: ': array([[3536,  464],
       [ 365, 3549]])}
{'F1 Score': 0.8954207140153905}


In [ ]:
# 3.4) SVM com stopwords ngram=1,1 
svm_classifier(vect_text, y_train, val_text, y_val)

SVM
{'Acurácia: ': 0.9020722769775082}
{'F1 Score': 0.9022082018927444}


In [ ]:
# 3.5) SVM sem stopwords 
svm_classifier(vect_text, y_train, val_text, y_val)

SVM
{'Acurácia: ': 0.9013141268637856}
{'F1 Score': 0.9013016555036016}


In [ ]:
# 3.6) Naive Bayes
naive_bayes_classifier(vect_text, y_train, val_text, y_val, alpha=1)

Naïve Bayes
{'Acurácia: ': 0.8685873136214304}
{'F1 Score': 0.8646538261322229}


### Conclusão CounterVectorize e TF-IDF

Com stopwords e unigramas:

- Arvore de Decisão: 70.16%
- Regressão Logística: 89.55%
- SVM: 90.22%
- Naïve Bayes: 86.46%

Sem stopwords (nltk) e unigramas:
- Regressão Logística: 89.54%
- SVM: 90.13%
- Naïve Bayes: 86.17%

Os modelos de Regressão Logistica e SVM sem retirar as stopwords do texto são os que se saíram melhor

### Modelo 3) Stemmer com unigrama

In [ ]:
check_module('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# transformar os dados de treino em numpy array
train = np.array(X_train)
train[0]

'sim e. na verdade, e em algum lugar no meu top 20 filmes favoritos de todos os tempos. numero 15, eu acho. de qualquer forma, eu geralmente nao sou de planos, mas acho que os enredos funcionam melhor em jogos de anime e rpg, final fantasy 7, por exemplo, e nao em filmes. mas este tem tudo. desenhos vividos de planetas, estrelas, um roteiro extremamente bem escrito. enquanto isso nao e realmente para criancas, eles ainda podem assistir, nao contem sangue, coragem e silicone. mas eu nao acho que eles vao entender isso.'

In [ ]:
# tokenizar os textos do dataset de treino
token_list = []
for text in tqdm(train):
    token_list.append(word_tokenize(text))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# visualizar tokenização de um texto
token_list[5]

['este',
 'filme',
 'nao',
 'e',
 'realmente',
 'um',
 'remake',
 'do',
 'filme',
 'de',
 '1949',
 'obrien',
 'que',
 'e',
 'excelente',
 '.',
 'ele',
 'empresta',
 'a',
 'premissa',
 'principal',
 '-',
 'um',
 'homem',
 'foi',
 'envenenado',
 'e',
 'passa',
 'o',
 'resto',
 'do',
 'filme',
 'tentando',
 'encontrar',
 'seu',
 'assassino',
 '.',
 'mas',
 'eu',
 'gosto',
 'que',
 'os',
 'escritores',
 'escolheram',
 'um',
 'professor',
 'de',
 'ingles',
 ',',
 'em',
 'vez',
 'de',
 'um',
 'penis',
 'particular',
 ',',
 'como',
 'protagonista',
 '.',
 'o',
 'enredo',
 'tambem',
 'e',
 'bastante',
 'original',
 '.',
 'em',
 'geral',
 ',',
 'o',
 'filme',
 'se',
 'move',
 'rapido',
 'o',
 'suficiente',
 'para',
 'mante-lo',
 'acordado',
 '.',
 'mas',
 'o',
 'que',
 'estraga',
 'este',
 'filme',
 'e',
 'uma',
 'qualidade',
 'estranha',
 'datada',
 'sobre',
 'ele',
 'provavelmente',
 'devido',
 'a',
 'horrenda',
 'musica',
 'original',
 'dos',
 'anos',
 '80',
 'scorecombined',
 'com',
 'uma',

In [ ]:
# transformar a lista dos textos tokenizados em numpy array
np_token_list = np.array(token_list)
np_token_list.shape

(31653,)

In [ ]:
# fazer o stemmatização do texto (pegar a raiz das palavras dos textos)
rslp = init_rslp()

stemmer_list = []
for token in tqdm(np_token_list):
    stemmer_list.append(stem_pandas(token))

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
# visualizar a stemmatização de um texto do dataset
stemmer_list[7]

"hom alon 3 e o prim do film hom alon a nao apresent culkin no papel princip e no mesm vilo . no entant , o enred e muit semelh ao film orig de hom alon . em vez de doi vilo comic , tem tre ou quatr del . est film envolv algum armadilh , mas tamb tem uma cen long com um carr de control remot . o hum pastela tamb e consist , mas o menin e os vilo real nao conseg caus impact ness film . nenhum trocadilh intenc . est film nao oferec nad de nov ou difer do que os film anteri fiz , e real nao ha o clim quent de fim de ano ou as subtram que os outr doi film tiv . e mais uma comed pur , mas nao consegu me faz rir , ja que os person real nao fiz iss por mim . eu nao recomend ess film ; e muit chat . se voc est procur um bom film famili com comed , enta assist ao film orig `` hom alon '' ."

In [ ]:
# transformar a lista dos textos stemmatizados em numpy array para entrada nas
# técnicas de TF-IDF
np_stemmer_list = np.array(stemmer_list)
np_stemmer_list.shape

(31653,)

In [ ]:
# Pipeline 
# TF-IDF
n_gram =(1,1)
#vect = tf_idf(ngram=n_gram, words=stopwords, idf=True)
vect = tf_idf(ngram=n_gram, words=None, idf=True)
fit = vect_fit(np_stemmer_list, vect)
vect_text = vect_transform(np_stemmer_list, fit)

val_text = vect_transform(X_val, fit)

In [ ]:
# modelo de árvore de decisao
decision_tree_classifier(vect_text, y_train, val_text, y_val)

Árvore de Decisão
{'Acurácia: ': 0.5875663381349507}
{'Matriz de Confusão: ': array([[1071, 2929],
       [ 335, 3579]])}
{'F1 Score': 0.6868163500287853}


In [ ]:
# modelo de regressão logistica
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Regressão Logística
{'Acurácia: ': 0.7505686125852918}
{'Matriz de Confusão: ': array([[2628, 1372],
       [ 602, 3312]])}
{'F1 Score': 0.7704117236566642}


### Conclusão Stemmer

Nenhum dos modelos gerados após stemmatização teve um bom desempenho comparando com os modelos gerados utilizando apenas TF-IDF sem stemmatizar.

O modelo de árvore de decisão teve F1 Score de 68.68%, **inferior ao F1 Score baseline**, que é de 69.88%. O modelo de regressão logística também ficou bem abaixo do melhor modelo gerado usando esta técnica de machine learning com F1 Score de 77.04%

### Modelo 4) Lemmer

- uso da biblioteca spacy, ja que a nltk não possui pos tagger para a lingua portuguesa
- comparação da quantidade de stopwords do nltk e spacy
- uso das stopwords das duas bibliotecas combinadas 

In [ ]:
# associar uma nova variável para manipular o dataset de treino
train = X_train

In [ ]:
counter = Counter()

In [ ]:
# instalar corpus de lingua portuguesa da biblioteca spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download pt

     |████████████████████████████████| 21.2MB 864kB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=850161589e45d5b5ae5a72d5497d19319cdae4dc76dacb5a289969adc7d0cc5c
  Stored in directory: /tmp/pip-ephem-wheel-cache-xu4qu84z/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [ ]:
# carregar corpus de stopwords do spacy (transformar em função)
nlp = spacy.load('pt')

In [ ]:
# juntar todos os tags de cada texto do dataset de treino 
# comando demorado para rodar (quase 33 minutos)
lista_pos = []

for text in tqdm(train):
    # documento do text0
    doc = nlp(text)
    # obter o pos tag do documento - vai pegar de todos os textos do dataset
    for token in doc:
        lista_pos.append(token.pos_)
        

/home/daniel/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [ ]:
# amostra da lista obtida no comando acima
lista_pos[:10]

['DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'VERB', 'NOUN', 'ADP', 'NOUN', 'PUNCT']

In [ ]:
# contar o pos tag de todo o texto
for pos in lista_pos:
    counter[pos] += 1

In [ ]:
counter

Counter({'DET': 1018423,
         'NOUN': 1432693,
         'CCONJ': 385947,
         'VERB': 1080735,
         'ADP': 921651,
         'PUNCT': 1053644,
         'PROPN': 665772,
         'ADJ': 515421,
         'NUM': 75667,
         'SYM': 89821,
         'ADV': 448996,
         'SCONJ': 156069,
         'PRON': 521800,
         'AUX': 156349,
         'X': 24474,
         'SPACE': 2984,
         'INTJ': 1144,
         'PART': 211})

In [ ]:
# transformar em dataframe
pos_tag = pd.DataFrame(data=counter.items(), columns=['Tag', 'Count']).sort_values(by='Count', ascending=False).reset_index()
pos_tag.drop(labels='index', axis=1, inplace=True)
pos_tag

,Tag,Count
0,NOUN,1432693
1,VERB,1080735
2,PUNCT,1053644
3,DET,1018423
4,ADP,921651
5,PROPN,665772
6,PRON,521800
7,ADJ,515421
8,ADV,448996
9,CCONJ,385947


In [ ]:
# salvar o postag em arquivo csv
pos_tag.to_csv('postag_filmes.csv', index=False)

O que siginificado de cada símbolo da lista de pos tag acima pode ser encontrada no link: https://spacy.io/api/annotation

In [ ]:
# função para lemmatizar todo o texto 
def lemmatizer_text(text):
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [ ]:
lemmatizer_text(train[1])

'este e um exemplo do motivar pelar qual o maioria dos filme de acao sao o mesmo . generico e chato , nao ha nado que valer o peno assistir aqui . um completar desperdicio dos talento de ice-t e cubar de gelar que ser mal aproveitar , cada um comprovar que sao capaz de atuar e agir bem . nao se incomodar com este , va ver new jack city , ricochet ou assistir new york undercover parir ice-t , ou boyz o hood , higher learning ou friday ser içar cubar e ver o negociar real . ice-ts horrivelmente cliche dialogar só fazer este filmar ralar o dente , e eu ainda estar me perguntar o que diabo bill paxton estar fazer n este filmar ? e por que diabo ele sempre interpretar exatamente o mesmo personagem ? dos extraterrestre em diante , todo o filme que eu vir com bill paxton o fazer interpretar exatamente o mesmo personagem irritante , e pelar menos em aliens seu personagem morrer , o que o tornar um pouco gratificante ... o geral , esse e lixar de acao de segundo classe . existir incontaveis    

In [ ]:
# lemmatização de todos os textos do dataset de treino - processo leva meia hora
# para processar
lemmatized_text = []

for text in tqdm(train):
    lemmatized_text.append(lemmatizer_text(text))
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
lemmatized_text[7]

'home alone 3 e o primeiro dos filme home alone o nao apresentar culkin o papel principal e o mesmo viloes . o entanto , o enredar e muito semelhante o o filmar original de home alone . em vez de dois viloes comicos , ter tres ou quatro d ele . este filmar envolver algum armadilhar , mas tambem ter umar cena longo com um carro de controlo remoto . o humor pastelao tambem e consistente , mas o menino e o viloes realmente nao conseguir causar impactar n esse filmar . nenhum trocadilhar intencional . este filmar nao oferecer nado de novo ou diferente do que o filme anterior fazer , e realmente nao ha o clima quentar de fim de ano ou o subtramas que o outro dois filme ter . e mais umar comedir puro , mas nao conseguir me fazer rir , ja que o personagem realmente nao fazer isso por mim . eu nao recomendar esse filmar ; e muito chato . se voce este procurar um bom filmar familiar com comedir , entao assistir o o filmar original " home alone " .'

In [ ]:
# colocar o array lemmatizado em um dataframe
texto = pd.DataFrame(lemmatized_text, columns=['texto_lemmatizado'])
# salvar dataset lemmatizado em arquivo csv
texto.to_csv('X_train_lemmatized.csv', index=False)

In [ ]:
# transformar o texto lemmatizado em np.array
lemmatized_text = np.array(lemmatized_text)

In [ ]:
# countvectorizer nas features de lemmatização e modelo
# Pipeline de modelagem

# usar unigramas
n_gram=(1,1)

# 1) Contagem de termos com unigrama sem retirar stopwords do texto
vect = count_term(ngram=n_gram, words=None)
fit = vect_fit(lemmatized_text, vect)
vect_text = vect_transform(lemmatized_text, fit)


# 2)transformar os dados de validação mantendo as regras usadas no  
# dataset de treino
val_text = vect_transform(X_val, fit)

# 3) Treinar modelo  
# 3.1) Árvore de decisão
decision_tree_classifier(vect_text, y_train, val_text, y_val)

# 3.2) Regressão Logística
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Árvore de Decisão
{'Acurácia: ': 0.6702047005307051}
{'Matriz de Confusão: ': array([[2499, 1501],
       [1109, 2805]])}
{'F1 Score': 0.6824817518248175}
Regressão Logística
{'Acurácia: ': 0.8217083649229214}
{'Matriz de Confusão: ': array([[2928, 1072],
       [ 339, 3575]])}
{'F1 Score': 0.8351828057469922}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# TF-IDF nas features lemmatizadas e modelo
# Pipeline
# 1) TD-IDF com unigrama sem retirar stopwords do texto
n_gram = (1,1)

vect = tf_idf(ngram=n_gram, words=stopwords, idf=True)
#vect = tf_idf(ngram=n_gram, words=None, idf=True)
fit = vect_fit(lemmatized_text, vect)
vect_text = vect_transform(lemmatized_text, fit)

# 2)transformar os dados de validação mantendo as regras usadas no  
# dataset de treino
val_text = vect_transform(X_val, fit)

# 3) Treinar modelo 
# 3.1) Árvore de decisão
decision_tree_classifier(vect_text, y_train, val_text, y_val)

# 3.2) Regressão Logística
logistic_regression_classifier(vect_text, y_train, val_text, y_val)

Árvore de Decisão
{'Acurácia: ': 0.6820823856456911}
{'Matriz de Confusão: ': array([[2614, 1386],
       [1130, 2784]])}
{'F1 Score': 0.6887679366650173}
Regressão Logística
{'Acurácia: ': 0.8569623452110184}
{'Matriz de Confusão: ': array([[3173,  827],
       [ 305, 3609]])}
{'F1 Score': 0.8644311377245509}


### Conclusão Lemmer
Sem retirar as stopwords do texto após lemmatizar tivemos um F1 Score de 69.92% no modelo de árvore de decisão e 85% no modelo de regressão logística. Após retirar as stopwords (corpus nltk) o modelo de árvore de decisão foi ainda pior com F1 Score de 68.20% enquanto a regressão logística se manteve próxima com 85.69%. Nenhum desses modelos melhorou o F1 Score obtido mais acima com TF-IDF sem retirar as stopwords do texto.

### Melhores modelos: Regressão logistica e SVM aplicando a técnica de **TF-IDF** e Unigrama **sem retirar** stopwords do texto

Hora da verdade: o F1 Score dos dois modelos citados acima foram os mais altos e relativamente próximos, portando selecionamos os dois para uma disputa final conferindo o F1 Score rodando todo o pipeline de transformação de cada um novamente, porém, ao invés de usar o dataset de validação usaremos agora o dataset de teste (como se fosse uma submissão no Kaggle). Ja sabemos que o modelo SVM leva muito tempo para rodar, enquanto a regressão logística é rápida, porém para a seleção do modelo final para este estudo o que será levado em consideração será de fato o F1 Score. 

In [ ]:
# Pipeline - dataset de teste
# 1) TD-IDF com unigrama retirarando stopwords do texto
n_gram = (1,1)
vect = tf_idf(ngram=n_gram, words=None, idf=True)
fit = vect_fit(X_train, vect)
vect_text = vect_transform(X_train, fit)

# 2)transformar os dados de validação mantendo as regras usadas no  
# dataset de treino
test_text = vect_transform(X_test, fit)

In [ ]:
# 3) Regressao logistica com os dados de teste
logistic_regression_classifier(vect_text, y_train, test_text, y_test)

Regressão Logística
{'Acurácia: ': 0.8807116862110796}
{'Matriz de Confusão: ': array([[4233,  652],
       [ 528, 4479]])}
{'F1 Score': 0.8836062339711975}


In [ ]:
# 4) SVM com os dados de test 
svm_classifier(vect_text, y_train, test_text, y_test)

SVM
{'Acurácia: ': 0.887484836231298}
{'F1 Score': 0.8905281794039539}


### Conclusão final:

O vencedor foi o modelo SVM com F1 Score de 89.05%. Este é o modelo em que deveremos fazer o deploy e colocar em produção.

### Apêndice

In [ ]:
# utlizamos neste notebook a biblioteca nltk para fazer transformações no texto com a finalidade de classificar este
# texto. A biblioteca spacy também possui um corpus de stopwords, portanto vamos olhar essas stopwords desta biblioteca

# numero de palavras no corpus
print('Stopwords spacy: ', len(nlp.Defaults.stop_words))

stops_nltk = list(nlp.Defaults.stop_words)

Stopwords spacy:  413


In [ ]:
# stopwords do SpaCy 
# examinando as stopwords deste corpus identificamos muitos termos que são comuns
# no português de Portugal, como no exemplo abaixo. Estas palavras raramente são
# encontradas em textos escritos no português Brasileiro, por isso todos os testes
# foram realizados utilizando somente o corpus de stopwords da biblioteca nltk
stops_nltk[:2]

['estivestes', 'fazeis']